In [12]:
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
# read in all the words

words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [14]:
# build hr vocab of charactersandmappings to/from integers
chars = sorted(list(set(''.join(words))))   
stoi = { ch:i for i,ch in enumerate(chars) }
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{0: '.', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}


In [15]:
#build the dataset

block_size = 3
X, Y = [], []

for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join([itos[i] for i in context]), '--->', itos[ix])
        context = context[1:] + [ix]


X = torch.tensor(X)
Y = torch.tensor(Y)


In [16]:
# karpathy making things respectable lol


In [17]:

X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [18]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [19]:
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True



3481


In [20]:
for _ in range(1000):
    # forward pass
    emb = C[X] # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
    logits = h @ W2 + b2 # (32, 27)
    loss = F.cross_entropy(logits, Y)
    print(loss.item())
    for p in parameters:
        p.grad = None
    loss.backward()
    #update
    for p in parameters:
        p.data += -0.1 * p.grad


19.646800994873047


In [ ]:
logits.max(1)

torch.return_types.max(
values=tensor([12.8706, 17.0122, 15.0809, 17.1209, 28.9917, 12.8706, 17.2103, 15.7414,
        18.6791, 18.9698, 22.7125, 23.3797, 12.8706, 12.8706, 18.9476, 20.9823,
        12.8706, 18.5093, 16.7913, 19.5030, 20.3325, 14.5317, 13.4665, 15.1977,
        22.2659, 12.8706, 19.4763, 17.1650, 11.6682, 14.8873, 26.1856, 24.1366],
       grad_fn=<MaxBackward0>),
indices=tensor([18, 12, 12,  0,  0, 18, 11,  8, 21,  8,  0,  0, 18, 18,  0,  0, 18, 18,
         0,  1,  4, 11, 11,  0,  0, 18, 14, 15,  7,  8,  0,  0]))

In [ ]:
Y

tensor([ 4, 12, 12,  0,  0, 14, 11,  8, 21,  8,  0,  0,  0, 21,  0,  0,  8, 18,
         0,  1,  4, 11, 11,  0,  0, 18, 14, 15,  7,  8,  0,  0])